In [29]:
import pandas as pd
from statsmodels.tsa.arima_model import ARIMA
import itertools
import datetime
import sys
import warnings
warnings.filterwarnings("ignore")

In [12]:
data = pd.read_excel('../data/service-data.xlsx')

In [13]:
data.head(2)

,Invoice Date,Job Card Date,Business Partner Name,Vehicle No.,Vehicle Model,Current KM Reading,INVOICE LINE TEXT
0,2017-05-30,2017-05-30,shivXXXXXXXXXX,KA03MFXXXX,BAJAJ AVENGER STREET 220,50000.0,ENGINE OIL
1,2017-06-02,2017-05-31,KIRAXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR NS 200,758.0,ENGINE OIL


In [14]:
data.dropna(inplace=True)

In [15]:
import datetime

datetime.datetime.strptime('2017-05-30', '%Y-%m-%d')

datetime.datetime(2017, 5, 30, 0, 0)

In [16]:
data.index = data['Invoice Date'].apply(lambda x: datetime.datetime.strptime(f'{x.year}-{x.month}', '%Y-%m'))

In [17]:
data[data['Vehicle Model'] == 'BAJAJ BOXER CT']

,Invoice Date,Job Card Date,Business Partner Name,Vehicle No.,Vehicle Model,Current KM Reading,INVOICE LINE TEXT
Invoice Date,,,,,,,
2017-06-01,2017-06-12,2017-06-12,jmkXXXXXXXXXX,KHXXXX,BAJAJ BOXER CT,3655.0,SPARK PLUG
2017-06-01,2017-06-12,2017-06-12,jmkXXXXXXXXXX,KHXXXX,BAJAJ BOXER CT,3655.0,AIR FILTER CHECKUP
2017-06-01,2017-06-12,2017-06-12,jmkXXXXXXXXXX,KHXXXX,BAJAJ BOXER CT,3655.0,GENERAL SERVICE
2017-06-01,2017-06-12,2017-06-12,jmkXXXXXXXXXX,KHXXXX,BAJAJ BOXER CT,3655.0,MIRROR ADJUSTMENT


In [18]:
data.head()

,Invoice Date,Job Card Date,Business Partner Name,Vehicle No.,Vehicle Model,Current KM Reading,INVOICE LINE TEXT
Invoice Date,,,,,,,
2017-05-01,2017-05-30,2017-05-30,shivXXXXXXXXXX,KA03MFXXXX,BAJAJ AVENGER STREET 220,50000.0,ENGINE OIL
2017-06-01,2017-06-02,2017-05-31,KIRAXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR NS 200,758.0,ENGINE OIL
2017-06-01,2017-06-02,2017-05-31,KIRAXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR NS 200,758.0,POLISH
2017-06-01,2017-06-02,2017-05-31,KIRAXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR NS 200,758.0,CONSUMABLES
2017-06-01,2017-06-02,2017-05-31,KIRAXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR NS 200,758.0,COOLANT OIL


In [19]:
value_counts = data['Vehicle Model'].value_counts()
value_counts

BAJAJ PULSAR 150            8614
BAJAJ AVENGER STREET 220    4212
BAJAJ PULSAR 220            2654
BAJAJ PULSAR 180            2251
BAJAJ PULSAR NS 200         2203
BAJAJ CT 100                1432
BAJAJ DISCOVER 125          1345
BAJAJ AVENGER STREET        1111
BAJAJ PLATINA                843
BAJAJ V150                   685
BAJAJ DISCOVER               678
BAJAJ PULSAR LS135           518
BAJAJ PULSAR AS 200          421
BAJAJ PULSAR NS 160          364
BAJAJ PULSAR RS 200          322
BAJAJ DISCOVER 150           282
BAJAJ AVENGER STREET 180     122
BAJAJ DISCOVER 135           115
BAJAJ V125                    95
BAJAJ V                       51
BAJAJ CALIBER                 40
BAJAJ XCD 125                 27
BAJAJ V15                     23
BAJAJ XCD 135                 13
WAVE                          10
BAJAJ DOMINAR 400 ABS          7
BAJAJ WIND 125                 6
BAJAJ BOXER CT                 4
Name: Vehicle Model, dtype: int64

In [20]:
value_counts.to_dict()

{'BAJAJ PULSAR 150': 8614,
 'BAJAJ AVENGER STREET 220': 4212,
 'BAJAJ PULSAR 220': 2654,
 'BAJAJ PULSAR 180': 2251,
 'BAJAJ PULSAR NS 200': 2203,
 'BAJAJ CT 100': 1432,
 'BAJAJ DISCOVER 125': 1345,
 'BAJAJ AVENGER STREET': 1111,
 'BAJAJ PLATINA': 843,
 'BAJAJ V150': 685,
 'BAJAJ DISCOVER': 678,
 'BAJAJ PULSAR LS135': 518,
 'BAJAJ PULSAR AS 200': 421,
 'BAJAJ PULSAR NS 160': 364,
 'BAJAJ PULSAR RS 200': 322,
 'BAJAJ DISCOVER 150': 282,
 'BAJAJ AVENGER STREET 180': 122,
 'BAJAJ DISCOVER 135': 115,
 'BAJAJ V125': 95,
 'BAJAJ V': 51,
 'BAJAJ CALIBER': 40,
 'BAJAJ XCD 125': 27,
 'BAJAJ V15': 23,
 'BAJAJ XCD 135': 13,
 'WAVE': 10,
 'BAJAJ DOMINAR 400 ABS': 7,
 'BAJAJ WIND 125': 6,
 'BAJAJ BOXER CT': 4}

In [21]:
prefix = 'Vehicle Model'
column = 'Vehicle Model'
data = pd.get_dummies(data, columns=[column], prefix=prefix)


In [24]:
dummy_columns = [i for i in data.columns if prefix in i]

dummy_columns_dict = {i: 'sum' for i in dummy_columns}


date_format = '%Y-%m-%d'
first_good_month = datetime.datetime.strptime('2017-06-01', date_format)
last_good_month = datetime.datetime.strptime('2018-12-31', date_format)
data = data [ (data.index >= first_good_month) & (data.index <= last_good_month)]

In [25]:
data_grouped = data[dummy_columns].groupby(data.index).agg(dummy_columns_dict)

In [30]:
def get_models(data_grouped, dummy_columns):
    models = {}
    for dummy_column in dummy_columns:
        data_for_model = data_grouped[dummy_column]
        r = range(0, 2)
        min_aic = sys.maxsize

        for p, d, q in itertools.product(r, r, r):
            try:
                model_arima = ARIMA(data_for_model, order=(p, d, q))
                model_arima_fit = model_arima.fit()
                pred = model_arima_fit.forecast(steps=9)[0]
                # print(p, d, q, pred, model_arima_fit.aic)
                if model_arima_fit.aic < min_aic:
                    result = (p, d, q, pred, model_arima_fit.aic)
                    min_aic = model_arima_fit.aic
            except:
                pass
                # print(p, d, q, 'bad')
        # print(invoice_line, result)
        models[dummy_column] = (model_arima, model_arima_fit)
    return models

models = get_models(data_grouped, dummy_columns)

In [31]:

models

{'Vehicle Model_BAJAJ AVENGER STREET': (<statsmodels.tsa.arima_model.ARIMA at 0x289caece240>,
  <statsmodels.tsa.arima_model.ARIMAResultsWrapper at 0x289caee5828>),
 'Vehicle Model_BAJAJ AVENGER STREET 180': (<statsmodels.tsa.arima_model.ARIMA at 0x289caece208>,
  <statsmodels.tsa.arima_model.ARIMAResultsWrapper at 0x289caee5fd0>),
 'Vehicle Model_BAJAJ AVENGER STREET 220': (<statsmodels.tsa.arima_model.ARIMA at 0x289c8669e80>,
  <statsmodels.tsa.arima_model.ARIMAResultsWrapper at 0x289caf162b0>),
 'Vehicle Model_BAJAJ BOXER CT': (<statsmodels.tsa.arima_model.ARIMA at 0x289caee5c18>,
  <statsmodels.tsa.arima_model.ARIMAResultsWrapper at 0x289caf16128>),
 'Vehicle Model_BAJAJ CALIBER': (<statsmodels.tsa.arima_model.ARIMA at 0x289caf16710>,
  <statsmodels.tsa.arima_model.ARIMAResultsWrapper at 0x289caf16f60>),
 'Vehicle Model_BAJAJ CT 100': (<statsmodels.tsa.arima_model.ARIMA at 0x289c2c1b978>,
  <statsmodels.tsa.arima_model.ARIMAResultsWrapper at 0x289c53c9fd0>),
 'Vehicle Model_BAJAJ D